In [1]:
import json
import re


In [2]:
from langchain.llms import Ollama

c:\Users\amanc\anaconda3\envs\ceras\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = Ollama(model='llama3.2')

C:\Users\amanc\AppData\Local\Temp\ipykernel_14732\3938246654.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='llama3.2')


In [4]:
import json
import re

# --- small safe wrapper to call your llm variable (handles common call styles) ---
def call_llm(prompt: str) -> str:
    try:
        out = llm(prompt)                 # callable
        if isinstance(out, str): return out
        if hasattr(out, 'text'): return out.text
        return str(out)
    except Exception:
        pass
    try:
        out = llm.generate(prompt)       # .generate()
        if isinstance(out, str): return out
        if isinstance(out, dict) and 'choices' in out:
            return ''.join(c.get('content','') for c in out['choices'])
        if hasattr(out, 'text'): return out.text
        return str(out)
    except Exception:
        pass
    try:
        out = llm.complete(prompt)       # .complete()
        if isinstance(out, str): return out
        if hasattr(out, 'text'): return out.text
        return str(out)
    except Exception:
        pass
    raise RuntimeError("Unable to call llm. Ensure `llm` is callable or supports .generate/.complete")



In [8]:
# --- decomposition prompt (concise, JSON-first) ---
DECOMP_PROMPT = (
    "You are a reasoning engine. Decompose the user's problem into a short ordered list "
    "of clear, atomic subtasks needed to solve it.\n"
    "Return output as JSON: {{ \"subtasks\": [ ... ] }} if possible. Be concise.\n"
    "User query: \"\"\"{query}\"\"\""
)

# --- function that returns list of subtasks only ---
def decompose_query(query: str) -> list:
    """
    Call the llm and return a list of subtasks.
    This version avoids KeyError by escaping literal braces in the prompt.
    """
    raw = call_llm(DECOMP_PROMPT.format(query=query))
    raw = (raw or "").strip()
    # Try JSON parse first
    try:
        obj = json.loads(raw)
        if isinstance(obj, dict) and 'subtasks' in obj and isinstance(obj['subtasks'], list):
            return [s.strip() for s in obj['subtasks'] if isinstance(s, str) and s.strip()]
    except Exception:
        pass
    # Try to find a JSON-like substring inside the model output
    try:
        jmatch = re.search(r'\{.*"subtasks".*\}', raw, flags=re.S)
        if jmatch:
            obj = json.loads(jmatch.group(0))
            if isinstance(obj, dict) and 'subtasks' in obj and isinstance(obj['subtasks'], list):
                return [s.strip() for s in obj['subtasks'] if isinstance(s, str) and s.strip()]
    except Exception:
        pass
    # Heuristic fallback: split numbered or dashed lists
    lines = []
    for line in raw.splitlines():
        line = line.strip()
        if not line:
            continue
        # drop common leading markers
        line = re.sub(r'^[0-9]+[)\.\-\s]+', '', line)
        line = re.sub(r'^[-*\u2022]\s*', '', line)
        # ignore long narrative lines that don't look like steps
        if 3 <= len(line) <= 300:
            lines.append(line)
    if lines:
        return lines
    # Last resort: return the original query as single step
    return [query.strip()]

In [9]:
# --- small runner that prints + returns subtasks ---
def run_decomposer(query: str):
    subtasks = decompose_query(query)
    print("Subtasks:")
    for i, s in enumerate(subtasks, 1):
        print(f" {i}. {s}")
    return subtasks

In [10]:
subtasks = run_decomposer("Write a Python script that downloads images from a URL list, resizes them to 512x512, and saves as PNG.")
print(subtasks)

C:\Users\amanc\AppData\Local\Temp\ipykernel_14732\1816498432.py:7: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  out = llm(prompt)                 # callable


Subtasks:
 1. Install necessary libraries (e.g., requests, Pillow)
 2. Create a list of URLs for images to download
 3. Download each image from URL and save as temporary file
 4. Use Pillow to resize each image to 512x512
 5. Convert resized image to PNG format and save
['Install necessary libraries (e.g., requests, Pillow)', 'Create a list of URLs for images to download', 'Download each image from URL and save as temporary file', 'Use Pillow to resize each image to 512x512', 'Convert resized image to PNG format and save']


In [12]:
subtasks = run_decomposer("how to make a plane engine faster?")
print(subtasks)

Subtasks:
 1. { "subtasks": [
 2. Identify the type of plane engine (e.g. turbofan, turboprop, jet engine),
 3. Research the current design specifications and performance characteristics of the engine,
 4. Explore aerodynamic and thermodynamic optimization techniques for increasing engine speed,
 5. Investigate advanced materials or manufacturing techniques that could improve engine durability and efficiency,
 6. Consider modifying the engine's compressor, turbine, or nozzle to increase airflow, pressure ratio, or exhaust velocity
 7. ] }
['{ "subtasks": [', 'Identify the type of plane engine (e.g. turbofan, turboprop, jet engine),', 'Research the current design specifications and performance characteristics of the engine,', 'Explore aerodynamic and thermodynamic optimization techniques for increasing engine speed,', 'Investigate advanced materials or manufacturing techniques that could improve engine durability and efficiency,', "Consider modifying the engine's compressor, turbine, or

In [14]:
subtasks = run_decomposer("how to develop an app in flutter")
print(subtasks)

Subtasks:
[]


In [11]:
import json, re

# Strong JSON-first prompt (use f-string to avoid brace escaping)
DECOMP_PROMPT_JSON = (
    "You are a reasoning engine. Decompose the user's problem into a short ordered list of clear, "
    "atomic subtasks needed to solve it. OUTPUT MUST BE STRICT JSON and nothing else, like:\n"
    '{"subtasks": ["step1", "step2", ...]}\n\n'
    "Be concise. Do not add commentary.\n\n"
    "User query:\n"
    "'''{query}'''\n"
)

DECOMP_PROMPT_SIMPLE = (
    "Decompose the user's request into a short ordered list of subtasks (plain text lines). "
    "Return only the list (one step per line). Keep steps atomic and actionable.\n\n"
    "User query:\n"
    "'''{query}'''\n"
)

def heuristic_fallback_for_query(query: str):
    """Deterministic fallback if LLM fails. Returns a reasonable generic decomposition for common tasks."""
    q = query.lower()
    # If it's about building an app (mobile/web) produce a standard flow
    if any(w in q for w in ("app", "application", "flutter", "react native", "android", "ios")):
        return [
            "Define app purpose, target users, and core features",
            "Design UI/UX sketches and basic navigation flow",
            "Choose tech stack and set up project (Flutter SDK, project structure)",
            "Implement core screens and navigation",
            "Implement data layer (local storage / API integration)",
            "Add assets and handle media/sizes",
            "Implement app logic and state management",
            "Test features locally (emulator / device) and fix bugs",
            "Prepare release (signing, build flavors) and deploy to app store / Play Store",
            "Plan post-release monitoring and updates"
        ]
    # If it's a generic 'how to' question give general research/decomposition steps
    return [
        "Clarify objective and desired outcome",
        "List required inputs and constraints",
        "Break the task into 3–6 atomic steps",
        "For each step, identify necessary tools or resources",
        "Execute steps in order, verify results after each",
        "Summarize and document the final result"
    ]

def parse_json_subtasks(raw: str):
    """Try to safely extract JSON substring with 'subtasks' and return list or None."""
    if not raw:
        return None
    raw = raw.strip()
    # try direct parse
    try:
        obj = json.loads(raw)
        if isinstance(obj, dict) and 'subtasks' in obj and isinstance(obj['subtasks'], list):
            return [s.strip() for s in obj['subtasks'] if isinstance(s, str) and s.strip()]
    except Exception:
        pass
    # try to find JSON-like block
    jmatch = re.search(r'\{[^}]*"subtasks"[^}]*\}', raw, flags=re.S)
    if jmatch:
        try:
            obj = json.loads(jmatch.group(0))
            if isinstance(obj, dict) and 'subtasks' in obj and isinstance(obj['subtasks'], list):
                return [s.strip() for s in obj['subtasks'] if isinstance(s, str) and s.strip()]
        except Exception:
            pass
    return None

def decompose_query(query: str) -> list:
    # 1) Try strict JSON output from LLM
    try:
        raw = call_llm(DECOMP_PROMPT_JSON.format(query=query))
    except Exception:
        raw = ""
    parsed = parse_json_subtasks(raw)
    if parsed:
        return parsed

    # 2) Retry with a simpler plain-list prompt
    try:
        raw2 = call_llm(DECOMP_PROMPT_SIMPLE.format(query=query))
    except Exception:
        raw2 = ""
    # try to extract lines from raw2
    lines = []
    if raw2:
        for line in raw2.splitlines():
            line = line.strip()
            if not line:
                continue
            # drop leading bullets/numbers
            line = re.sub(r'^[0-9]+[)\.\-\s]+', '', line)
            line = re.sub(r'^[-*\u2022]\s*', '', line)
            if 3 <= len(line) <= 300:
                lines.append(line)
    if lines:
        return lines

    # 3) Deterministic heuristic fallback based on query
    return heuristic_fallback_for_query(query)

def run_decomposer(query: str):
    subtasks = decompose_query(query)
    print("Subtasks:")
    for i, s in enumerate(subtasks, 1):
        print(f" {i}. {s}")
    return subtasks

# --- quick local test (you can run this) ---
# subtasks = run_decomposer("how to develop an app in flutter")
# print(subtasks)


In [17]:
subtasks = run_decomposer("how to develop an app in flutter")
print(subtasks)

Subtasks:
 1. Define app purpose, target users, and core features
 2. Design UI/UX sketches and basic navigation flow
 3. Choose tech stack and set up project (Flutter SDK, project structure)
 4. Implement core screens and navigation
 5. Implement data layer (local storage / API integration)
 6. Add assets and handle media/sizes
 7. Implement app logic and state management
 8. Test features locally (emulator / device) and fix bugs
 9. Prepare release (signing, build flavors) and deploy to app store / Play Store
 10. Plan post-release monitoring and updates
['Define app purpose, target users, and core features', 'Design UI/UX sketches and basic navigation flow', 'Choose tech stack and set up project (Flutter SDK, project structure)', 'Implement core screens and navigation', 'Implement data layer (local storage / API integration)', 'Add assets and handle media/sizes', 'Implement app logic and state management', 'Test features locally (emulator / device) and fix bugs', 'Prepare release (s

In [16]:
subtasks = run_decomposer("how high is the sky")
print(subtasks)

Subtasks:
 1. Clarify objective and desired outcome
 2. List required inputs and constraints
 3. Break the task into 3–6 atomic steps
 4. For each step, identify necessary tools or resources
 5. Execute steps in order, verify results after each
 6. Summarize and document the final result
['Clarify objective and desired outcome', 'List required inputs and constraints', 'Break the task into 3–6 atomic steps', 'For each step, identify necessary tools or resources', 'Execute steps in order, verify results after each', 'Summarize and document the final result']


In [4]:
from llm_utils import run_decomposer

subtasks = run_decomposer("what is (a^2 - b^2)? how can i solve it effeciently?")
print(subtasks)

Subtasks:
 1. Expand the expression using difference of squares formula: a^2 - b^2 = (a + b)(a - b)
 2. Factorize and simplify the result if necessary
 3. Determine the most efficient method to solve algebraic expressions, such as factoring or using formulas
 4. Apply the determined method to expand and factorize the expression
['Expand the expression using difference of squares formula: a^2 - b^2 = (a + b)(a - b)', 'Factorize and simplify the result if necessary', 'Determine the most efficient method to solve algebraic expressions, such as factoring or using formulas', 'Apply the determined method to expand and factorize the expression']
